# Exciton Decomposition Analysis

There are qualititative and quantative analysis, for qualititive you can use summarize eigenvectors, just know the largest band to band transition and largest k point, then you will learn the band composition into parts.

For ploteigenvectors.py is also a qualititive analysis, summarize eigenvectors is also semi-qualiative, because of limited information: 

It only tells you about wtot, wmax and ikmax for different band to band transitions for each Exciton S - You cannot resolve K-point from this information, thus cannot safely project to atomic orbitals.

`wtot = sum_k |A_vck|^2. wmax = max_k |A_vck|^2. |A_vc (ikmax)|^2 = wmax.`


**To do a proper quantitative approach:**

Read eigenvectors.h5 to extract K resolved decomposition into different band to band transition, in combination with pdos information, I can calculate the percentage for perovskite/non-perovskite of electrons and holes for the every excitons, similar information are also calculated when I did the ECF-noshift plot, where I can see the localization of electrons and holes for each exciton in real space.

file:///Users/yinanchen/PhD/2022_Intergrowth_Hema/APD1_Na/absorption/k104/evec_component_k=0.png
file:///Users/yinanchen/PhD/2022_Intergrowth_Hema/APD1_Na/absorption/k104/evec_component_k=-1.png
file:///Users/yinanchen/PhD/2022_Intergrowth_Hema/APD2_Pb/corrected_H/absorption/evec_component_k=-1.png

In [242]:
import pandas as pd
import numpy as np

In [243]:
# Things to modify by hand

kgrid_file = 'kgrid.log_cocn.bak'
kgrid_file = 'kgrid.log_CYS.bak'
kgrid_file = 'kgrid.log_APD2.bak'
kgrid_file = 'kgrid.log_APD1.bak'

bandstructure_file = 'bandstructure.dat_cocn.bak'
bandstructure_file = 'bandstructure.dat_CYS.bak'
bandstructure_file = 'bandstructure.dat_APD2.bak'
bandstructure_file = 'bandstructure.dat_APD1.bak'

fermi = 280 # ! for HOCN
fermi = 304   # ! for COCN
fermi = 544   # ! for CYS
fermi = 408 # ! VBM - start + 1 , hard-coded for APD2_Li now, read from projbands file
fermi = 432   # ! for APD1_Na


eigenvector_file = 'eigenvectors.h5_hocn.bak'
eigenvector_file = 'eigenvectors.h5_cocn.bak'
eigenvector_file = 'eigenvectors.h5.CYS.bak'
eigenvector_file = 'eigenvectors.h5.APD2.bak'
eigenvector_file = 'eigenvectors.h5.APD1.bak'

scf_file = 'relax.out_cocn.bak'
scf_file = 'scf.out_CYS.bak'
scf_file = 'scf.out_APD2_Li.bak'
scf_file = 'scf.out_APD1.bak'

projbands_file = 'cl.projbands_hocn.bak'
projbands_file = 'cl.projbands_cocn.bak'
projbands_file = 'cl.projbands.CYS.bak'
projbands_file = 'cl.projbands_APD2_Li.bak'
projbands_file = 'cl.projbands.apd1.bak'
e_min = 0
e_high = 5

## Step 0: Find the K list correspondance from bandstructure.dat (unfolded) with the K list from kgrid.log (folded)

### 0.1 Process K list from Kgrid.log (folded)

In [244]:
## Process the kgrid.log, find the correpondance with uniform grid and irrduciible grid used to calculate WFN_fi

def process_kgrid_WFN(file_content):
    lines = file_content.split('\n')

    for i, line in enumerate(lines):
        if "k-points in the original uniform grid" in line:
            # Next line contains the number of rows to read
            num_rows = int(lines[i + 1].strip())
            # Data starts from the next line
            start_line = i + 2
            break

    data = []
    for line in lines[start_line:start_line + num_rows]:
        # Split the line into columns and convert to appropriate types
        columns = line.split()
        row = [int(columns[0])] + [float(c) for c in columns[1:5]] + [int(columns[5]), columns[6]]
        data.append(row)
    unfolded_idx = {}
    
    cnt = 0
    processed_data = []

    for row in data:
        row_number = row[0]
        reference_row = row[5]

        # If the sixth column is 0, increment the count
        if reference_row == 0:
            unfolded_idx[row_number] = cnt
            cnt = cnt + 1
        else:
            # If the sixth column is not 0, set the count to the count of the referenced row
            unfolded_idx[row_number] = unfolded_idx[reference_row]
        processed_data.append(row + [unfolded_idx[row_number]])

    return processed_data

processed_data = process_kgrid_WFN(open(kgrid_file).read())
# the last column and kx, ky, kz
processed_data = np.array(processed_data)[:, [1, 2, 3, 7]].astype(float)
nk = processed_data.shape[0]
processed_data

array([[0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00],
       [0.00e+00, 0.00e+00, 2.50e-01, 1.00e+00],
       [0.00e+00, 0.00e+00, 5.00e-01, 2.00e+00],
       ...,
       [9.00e-01, 9.00e-01, 2.50e-01, 3.97e+02],
       [9.00e-01, 9.00e-01, 5.00e-01, 3.98e+02],
       [9.00e-01, 9.00e-01, 7.50e-01, 3.99e+02]])

### 0.2 Process bandstructure.dat (unfolded)

In [245]:
def read_unfolded(file_path):
    matrix = []
    last_spin, last_band = None, None

    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('#'):
                continue

            parts = line.split()
            if len(parts) < 5:
                continue  
            spin, band, kx, ky, kz = parts[:5]

            # check if we have a new spin or band
            if last_spin is not None and (spin != last_spin or band != last_band):
                break
            # update last spin and band
            last_spin, last_band = spin, band

            matrix.append([float(kx), float(ky), float(kz)])

    return np.array(matrix)

data_matrix = read_unfolded(bandstructure_file)
print(data_matrix.shape)


(400, 3)


In [246]:

#reciprocal_lattice_vectors and alat are read from relax.out or scf.out

def extract_specific_data(file_path):
    with open(file_path, 'r') as file:
        alat = None
        b_vectors = []

        for line in file:
            if 'lattice parameter (alat)' in line:
                parts = line.split('=')
                if len(parts) >= 2:
                    alat = float(parts[1].split()[0])

            if 'reciprocal axes' in line:
                for i in range(3):
                    b_vectors.append(file.readline().split()[3:6])
            # ! Heavily depends on the format of the file
            if alat is not None and len(b_vectors) == 3:
                break

    return alat, np.array(b_vectors, dtype=float)

alat, b_vectors = extract_specific_data(scf_file)
print(alat)
print(b_vectors)
two_pi_over_alat = 2 * np.pi / alat
reciprocal_lattice_vectors = np.array(b_vectors) * two_pi_over_alat
reciprocal_lattice_vectors

15.4338
[[ 1.       -0.        0.040671]
 [ 0.        1.05533  -0.      ]
 [ 0.        0.        0.349423]]


array([[ 0.40710553, -0.        ,  0.01655739],
       [ 0.        ,  0.42963068, -0.        ],
       [ 0.        ,  0.        ,  0.14225204]])

In [247]:
# Convert cartesian coordinates to fractional coordinates
def cartesian_to_fractional(cartesian_coordinates, reciprocal_lattice_vectors):
    return np.dot(cartesian_coordinates, np.linalg.inv(reciprocal_lattice_vectors))

def fractional_to_cartesian(fractional_coordinates, reciprocal_lattice_vectors):
    return np.dot(fractional_coordinates, reciprocal_lattice_vectors)

fractional_coordinates = cartesian_to_fractional(data_matrix, reciprocal_lattice_vectors)
# Around to 3 decimals
fractional_coordinates = np.around(fractional_coordinates, decimals=3)
fractional_coordinates

array([[ 0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.05,  0.  ],
       [ 0.  ,  0.1 ,  0.  ],
       ...,
       [ 0.5 , -0.15,  0.  ],
       [ 0.5 , -0.1 ,  0.  ],
       [ 0.5 , -0.05,  0.  ]])

In [248]:
# Convert fractional coordinates to the BZ within (0,0,0) to (1,1,1) by translating the coordinates
def translate_to_BZ(fractional_coordinates):
    return fractional_coordinates - np.floor(fractional_coordinates)

translated_coordinates = translate_to_BZ(fractional_coordinates)
translated_coordinates[:20]

array([[0.  , 0.  , 0.  ],
       [0.  , 0.05, 0.  ],
       [0.  , 0.1 , 0.  ],
       [0.  , 0.15, 0.  ],
       [0.  , 0.2 , 0.  ],
       [0.  , 0.25, 0.  ],
       [0.  , 0.3 , 0.  ],
       [0.  , 0.35, 0.  ],
       [0.  , 0.4 , 0.  ],
       [0.  , 0.45, 0.  ],
       [0.  , 0.5 , 0.  ],
       [0.  , 0.55, 0.  ],
       [0.  , 0.6 , 0.  ],
       [0.  , 0.65, 0.  ],
       [0.  , 0.7 , 0.  ],
       [0.  , 0.75, 0.  ],
       [0.  , 0.8 , 0.  ],
       [0.  , 0.85, 0.  ],
       [0.  , 0.9 , 0.  ],
       [0.  , 0.95, 0.  ]])

In [249]:
# Sort the translated coordinates first by kx, then by ky, then by kz, and keeping the original indices
def sort_by_k(translated_coordinates):
    indices = np.lexsort((translated_coordinates[:,2], translated_coordinates[:,1], translated_coordinates[:,0]))
    return translated_coordinates[indices], indices

sorted_coordinates, indices = sort_by_k(translated_coordinates)
print(sorted_coordinates[:20])
print(indices[:20])

[[0.   0.   0.  ]
 [0.   0.05 0.  ]
 [0.   0.1  0.  ]
 [0.   0.15 0.  ]
 [0.   0.2  0.  ]
 [0.   0.25 0.  ]
 [0.   0.3  0.  ]
 [0.   0.35 0.  ]
 [0.   0.4  0.  ]
 [0.   0.45 0.  ]
 [0.   0.5  0.  ]
 [0.   0.55 0.  ]
 [0.   0.6  0.  ]
 [0.   0.65 0.  ]
 [0.   0.7  0.  ]
 [0.   0.75 0.  ]
 [0.   0.8  0.  ]
 [0.   0.85 0.  ]
 [0.   0.9  0.  ]
 [0.   0.95 0.  ]]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


In [250]:
# append the indices to the processed_data
processed_kgrid_wfn_eigenvector = np.hstack((processed_data, indices.reshape(-1,1)))
processed_kgrid_wfn_eigenvector[:20]

array([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.25,  1.  ,  1.  ],
       [ 0.  ,  0.  ,  0.5 ,  2.  ,  2.  ],
       [ 0.  ,  0.  ,  0.75,  3.  ,  3.  ],
       [ 0.  ,  0.1 ,  0.  ,  4.  ,  4.  ],
       [ 0.  ,  0.1 ,  0.25,  5.  ,  5.  ],
       [ 0.  ,  0.1 ,  0.5 ,  6.  ,  6.  ],
       [ 0.  ,  0.1 ,  0.75,  7.  ,  7.  ],
       [ 0.  ,  0.2 ,  0.  ,  8.  ,  8.  ],
       [ 0.  ,  0.2 ,  0.25,  9.  ,  9.  ],
       [ 0.  ,  0.2 ,  0.5 , 10.  , 10.  ],
       [ 0.  ,  0.2 ,  0.75, 11.  , 11.  ],
       [ 0.  ,  0.3 ,  0.  , 12.  , 12.  ],
       [ 0.  ,  0.3 ,  0.25, 13.  , 13.  ],
       [ 0.  ,  0.3 ,  0.5 , 14.  , 14.  ],
       [ 0.  ,  0.3 ,  0.75, 15.  , 15.  ],
       [ 0.  ,  0.4 ,  0.  , 16.  , 16.  ],
       [ 0.  ,  0.4 ,  0.25, 17.  , 17.  ],
       [ 0.  ,  0.4 ,  0.5 , 18.  , 18.  ],
       [ 0.  ,  0.4 ,  0.75, 19.  , 19.  ]])

In [251]:
# Check both indices are correct, the first one should be foled index, the second one is the unfolded index
print(max(processed_kgrid_wfn_eigenvector[:,4]))
print(max(processed_kgrid_wfn_eigenvector[:,3]))
print(len(processed_kgrid_wfn_eigenvector))

399.0
399.0
400


## Step 1: Decomposition of excitons into the contribution of band-to-band transitions at different k points

Specifically, I read and store sum_v/c |Acvk|^2 for each (S,k) pair for each band in c, v respectively. Then I can also plot the distribution of the contribution of each band-to-band transition to the exciton wavefunction as I did using the script plot_eigenvectors_kloop.py.

file:///Users/yinanchen/PhD/2022_Intergrowth_Hema/APD1_Na/absorption/k104/evec_component_k=0.png
file:///Users/yinanchen/PhD/2022_Intergrowth_Hema/APD1_Na/absorption/k104/evec_component_k=-1.png

In [252]:
with open(projbands_file, 'r') as f:
    lines = f.readlines()

comments = []
for line in lines:
    if line.startswith('#'):
        comments.append(line)
    else:
        break

data = []
for line in lines:
    if not line.startswith('#'):
        data.append(line)


In [253]:
# convert the list of strings to a list of lists, 2D array
data = [line.split() for line in data]

In [254]:
# conver the 2d array to a dataframe, and convert the strings to floats
df = pd.DataFrame(data).astype(float)

In [255]:
df

,0,1,2,3,4,5,6,7,8,9,...,674,675,676,677,678,679,680,681,682,683
0,1.0,0.0000,-51.14744,0.996,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,1.0,0.0000,-51.14744,0.996,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,1.0,0.0000,-51.14743,0.996,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,1.0,0.0000,-51.14743,0.996,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,1.0,0.0000,-51.14737,0.996,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182395,400.0,39.1237,4.41635,0.728,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.011,0.011,0.000,0.000,0.000,0.000,0.000,0.000
182396,400.0,39.1237,4.42564,0.824,0.0,0.0,0.0,0.0,0.0,0.0,...,0.005,0.005,0.013,0.013,0.007,0.003,0.003,0.007,0.005,0.005
182397,400.0,39.1237,4.43226,0.824,0.0,0.0,0.0,0.0,0.0,0.0,...,0.005,0.005,0.012,0.012,0.006,0.003,0.003,0.006,0.005,0.005
182398,400.0,39.1237,4.54712,0.808,0.0,0.0,0.0,0.0,0.0,0.0,...,0.005,0.005,0.011,0.011,0.008,0.003,0.003,0.008,0.005,0.005


In [256]:
import numpy as np
import h5py as h5
import matplotlib.pyplot as plt
contrib_c_dict = {}
contrib_v_dict = {}

def plot_for_k(k_select):
    f = h5.File(eigenvector_file,'r')
    evals = f['exciton_data/eigenvalues'][()]
    evc = f['exciton_data/eigenvectors'][()]
    evc = evc[0,:,:,:,:,0,:]
    (nS, nk, nc, nv, _) = evc.shape

    if k_select == -1:
        pass
    else:
        evc = evc[:,[k_select],:,:,:]

    for iN_S in range(nS):
        if e_min<evals[iN_S]<e_high:
            temp_contrib_cv = np.sum(abs(evc[iN_S,:,:,:,0]+evc[iN_S,:,:,:,1]*1j)**2, axis=0) # Summarize_eigenvectors and Ploteigenvectors - For a certain k and c, calculating the sum of Acvk across all vs, and vice versa, this is the correct way to do it, because for a certain k, valence band pdos idoes not change when we fix a v, and vice versa, But you ignore the correlation between c and v, thus you miss the ECF, but you can only plot ECF_noshift
            temp_contrib_v = np.sum(temp_contrib_cv, axis=0)
            temp_contrib_c = np.sum(temp_contrib_cv, axis=1)

            contrib_c_dict[(iN_S,k_select)] = temp_contrib_c
            contrib_v_dict[(iN_S,k_select)] = temp_contrib_v
    return nS, nk, nc, nv

In [257]:
for k in range(0,nk):
    nS, nk, nc, nv = plot_for_k(k)
    #print(contrib_c_dict)
    #print(contrib_v_dict)
print(nk)

400


In [258]:
contrib_c_dict

{(0,
  0): array([7.71260819e-03, 6.51098749e-03, 1.03275489e-04, 1.06261732e-04,
        2.36370929e-08, 2.20814838e-08, 5.18146574e-09, 1.94460114e-09,
        9.67786800e-06, 9.06592567e-06, 5.66956163e-08, 1.92217435e-07,
        5.81337458e-06, 5.20091112e-06, 1.64793501e-08, 2.11270789e-08,
        6.68127998e-07, 2.48294597e-07, 1.24500326e-06, 1.29891253e-06]),
 (1,
  0): array([7.54194011e-03, 6.68528206e-03, 8.00968458e-05, 1.21456231e-04,
        1.45627943e-08, 1.57299205e-08, 8.07810392e-09, 5.67648178e-09,
        5.37481470e-06, 8.21762272e-06, 4.14893024e-06, 9.75173854e-06,
        6.13716690e-06, 1.79301768e-06, 1.15297583e-08, 1.21332817e-08,
        6.72906179e-07, 1.09824674e-06, 1.18161897e-06, 5.87927821e-07]),
 (2,
  0): array([8.28727556e-03, 9.33097174e-03, 1.47392930e-04, 9.31907118e-05,
        1.18224478e-08, 1.11934038e-08, 7.82916274e-10, 4.39511146e-09,
        6.15766812e-06, 4.06123775e-06, 1.53427559e-05, 8.01053970e-06,
        4.06846524e-07, 4.9850

In [259]:
nS, nk, nc, nv

(10, 400, 20, 20)

In [260]:
# find the corresponding pdos for each k from df
# Count the number of k points in df, should be consistent with nk
n_k = len(df[0].unique())
n_k

400

## Step 2: Calculate the kpdos on a WFN_fi calculation

Now that we have the decomposition of excitons into the contribution of band-to-band transitions for each (S,k) pair.

For every k, find the corresponding pdos from K-resolved Projected DOS on WFN_fi.

In [261]:
# Before that, we need to isolating only the middle nv+nc rows for each k, 
# corresponding to the nv valence and nc conduction bands

# First group df by the first column, then operate on each group
# For each group, take out the middle nv+nc rows around fermi, i.e. fermi - nv : fermi + nc
# ! Locate the position of fermi, i.e. the position where df[2] changes from negative to positive. 

conduction_df = df.groupby(0).apply(lambda x: x.iloc[fermi:fermi+nc,:])
valence_df = df.groupby(0).apply(lambda x: x.iloc[fermi-nv:fermi,:])
reversed_valence_df = valence_df.iloc[::-1,:]
reversed_valence_df

0        1        2      3    4    5    6    7    8    9    \
0                                                                            
400.0 182375  400.0  39.1237 -0.01134  0.956  0.0  0.0  0.0  0.0  0.0  0.0   
      182374  400.0  39.1237 -0.02349  0.956  0.0  0.0  0.0  0.0  0.0  0.0   
      182373  400.0  39.1237 -0.16839  0.920  0.0  0.0  0.0  0.0  0.0  0.0   
      182372  400.0  39.1237 -0.18269  0.904  0.0  0.0  0.0  0.0  0.0  0.0   
      182371  400.0  39.1237 -0.46839  0.872  0.0  0.0  0.0  0.0  0.0  0.0   
...             ...      ...      ...    ...  ...  ...  ...  ...  ...  ...   
1.0   416       1.0   0.0000 -1.31428  0.884  0.0  0.0  0.0  0.0  0.0  0.0   
      415       1.0   0.0000 -1.32213  0.952  0.0  0.0  0.0  0.0  0.0  0.0   
      414       1.0   0.0000 -1.32213  0.952  0.0  0.0  0.0  0.0  0.0  0.0   
      413       1.0   0.0000 -1.32498  0.872  0.0  0.0  0.0  0.0  0.0  0.0   
      412       1.0   0.0000 -1.32498  0.872  0.0  0.0  0.0  0.0  0.0  0.0   

              ...  674  675  676  677  678  679  680  681  682  683  
0             ...                                                    
400.0 182375  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
      182374  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
      182373  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
      182372  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
      182371  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
...           ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  
1.0   416     ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
      415     ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
      414     ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
      413     ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
      412     ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[8000 rows x 684 columns]

In [262]:
# This step is to check the vs are negative and cs are positive
# To ensure that the hard-coded fermi is correct

valence_df.loc[1]
print(valence_df.loc[1][2].values)
print(reversed_valence_df.loc[1][2].values)
print(conduction_df.loc[1][2].values)

[-1.32498e+00 -1.32498e+00 -1.32213e+00 -1.32213e+00 -1.31428e+00
 -1.31428e+00 -1.21974e+00 -1.21974e+00 -1.16746e+00 -1.16746e+00
 -1.09867e+00 -1.09867e+00 -1.08051e+00 -1.08051e+00 -4.70260e-01
 -4.70260e-01 -1.64410e-01 -1.64410e-01  1.00000e-05  1.00000e-05]
[ 1.00000e-05  1.00000e-05 -1.64410e-01 -1.64410e-01 -4.70260e-01
 -4.70260e-01 -1.08051e+00 -1.08051e+00 -1.09867e+00 -1.09867e+00
 -1.16746e+00 -1.16746e+00 -1.21974e+00 -1.21974e+00 -1.31428e+00
 -1.31428e+00 -1.32213e+00 -1.32213e+00 -1.32498e+00 -1.32498e+00]
[2.39009 2.39009 2.40832 2.40832 3.40609 3.40609 3.65698 3.65698 3.74319
 3.74319 3.75925 3.75925 3.87811 3.87811 4.22091 4.22091 4.29741 4.29741
 4.40126 4.40126]


In [263]:
# To correspond to the contribution of valence bands (counting from the VBM) and conduction bands (counting from the CBM)
# We need to reverse the order of the valence bands

reversed_valence_df.loc[41]
conduction_df.loc[41]


,0,1,2,3,4,5,6,7,8,9,...,674,675,676,677,678,679,680,681,682,683
18672,41.0,4.01275,2.16489,0.948,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
18673,41.0,4.01275,2.33119,0.924,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
18674,41.0,4.01275,2.58664,0.924,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
18675,41.0,4.01275,2.70301,0.940,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
18676,41.0,4.01275,3.42275,0.836,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
18677,41.0,4.01275,3.43927,0.848,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
18678,41.0,4.01275,3.54247,0.900,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
18679,41.0,4.01275,3.61251,0.904,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
18680,41.0,4.01275,3.69647,0.764,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
18681,41.0,4.01275,3.75902,0.745,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [264]:
# Normalize the reversed_valence_df and conduction_df by the sum of the pdos of each k (the 3rd column)

reversed_valence_df = reversed_valence_df.apply(lambda x: x/x[3], axis=1)
conduction_df = conduction_df.apply(lambda x: x/x[3], axis=1)
reversed_valence_df.loc[41]
conduction_df.loc[41]

,0,1,2,3,4,5,6,7,8,9,...,674,675,676,677,678,679,680,681,682,683
18672,43.248945,4.232859,2.283639,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
18673,44.372294,4.342803,2.522933,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
18674,44.372294,4.342803,2.799394,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
18675,43.617021,4.268883,2.875543,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
18676,49.043062,4.799940,4.094199,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
18677,48.349057,4.732017,4.055743,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
18678,45.555556,4.458611,3.936078,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
18679,45.353982,4.438883,3.996139,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
18680,53.664921,5.252291,4.838312,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
18681,55.033557,5.386242,5.045664,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Step 3: Combine the knowledge of K resolved pdos and exciton decomposition into bands

For each k point, find out the decomposition of bands into atomic orbitals

In [265]:
# Now we use contrib_v_dict[(s,k)] as the weight to multiply with the pdos of the valence band at k = k_select
# Then we sum over all k points, and we get the pdos of the valence band

# ! Note, Question: should we normalize the weight? i.e. divided by the sum of the weights over all vs or cs for each k point?
# ! Answer: No, because each k point in BZ should have a different total weight as seen by the plot_eigenvectors.py plots,
# #! so we should not normalize the weight for each k point.

# ! Also, I am not normalizing pdos (not divided by TOTAL (ranging from 0.7 - 0.9) in column 4), 
# ! using their original pdos at (k, n), where n is the band index.
# ! We should normalize the pdos by the TOTAL

s_select = 9
result_df_valence = pd.DataFrame()
result_df_conduction = pd.DataFrame()
for k_select in range(len(processed_kgrid_wfn_eigenvector)):
    k_select_kpdos =  int(processed_kgrid_wfn_eigenvector[k_select][3])
    k_select_eigenvector = int(processed_kgrid_wfn_eigenvector[k_select][4])
    weighted_sum_valence = reversed_valence_df.loc[k_select_kpdos+1].mul(contrib_v_dict[(s_select,k_select_eigenvector)], axis=0).sum()
    # \ / contrib_v_dict[(s_select,k_select)].sum()
    result_df_weighted_sum_valence = pd.DataFrame([weighted_sum_valence])

    weighted_sum_conduction = conduction_df.loc[k_select_kpdos+1].mul(contrib_c_dict[(s_select,k_select_eigenvector)], axis=0).sum()
    # \ / contrib_c_dict[(s_select,k_select)].sum()
    result_df_weighted_sum_conduction = pd.DataFrame([weighted_sum_conduction])

    # append all k points into a single dataframe
    #result_df = result_df.append(result_df_weighted_sum)
    result_df_valence = pd.concat([result_df_valence, result_df_weighted_sum_valence], axis=0)
    result_df_conduction = pd.concat([result_df_conduction, result_df_weighted_sum_conduction], axis=0)

# sum over all k points, row-wise
print(result_df_valence.sum(axis=0))
print(result_df_conduction.sum(axis=0))
result_df_conduction

0      74.492674
1       6.802828
2      -0.352518
3       1.000000
4       0.000000
         ...    
679     0.000000
680     0.000000
681     0.000000
682     0.000000
683     0.000000
Length: 684, dtype: float64
0      7.202835e+01
1      6.579028e+00
2      2.975912e+00
3      1.000000e+00
4      0.000000e+00
           ...     
679    6.601036e-08
680    6.604665e-08
681    1.793859e-07
682    1.147383e-07
683    1.146511e-07
Length: 684, dtype: float64


,0,1,2,3,4,5,6,7,8,9,...,674,675,676,677,678,679,680,681,682,683
0,0.019757,0.000000,0.047225,0.018568,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.000000e+00,3.832311e-10,3.832311e-10,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00
0,0.037910,0.001000,0.045320,0.017480,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.000000e+00,5.242911e-10,5.242911e-10,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00
0,0.058881,0.002071,0.048036,0.018679,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.000000e+00,5.671121e-10,5.671121e-10,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00
0,0.072127,0.002854,0.044643,0.016914,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.000000e+00,1.637817e-09,1.637817e-09,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00
0,0.020903,0.000882,0.010934,0.003913,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.000000e+00,3.850171e-10,3.850171e-10,2.746649e-11,0.0,0.0,2.746649e-11,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.388640,0.038189,0.002810,0.000907,0.0,0.0,0.0,0.0,0.0,0.0,...,4.675400e-11,4.675400e-11,5.964728e-09,5.964728e-09,5.079978e-10,0.0,0.0,5.079978e-10,4.675400e-11,4.675400e-11
0,0.287011,0.028170,0.001984,0.000665,0.0,0.0,0.0,0.0,0.0,0.0,...,6.032511e-11,6.032511e-11,4.052945e-09,4.052945e-09,5.937363e-10,0.0,0.0,5.937363e-10,6.032511e-11,6.032511e-11
0,0.378458,0.037102,0.002518,0.000871,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.000000e+00,1.558255e-09,1.558255e-09,3.805502e-10,0.0,0.0,3.805502e-10,0.000000e+00,0.000000e+00
0,0.032244,0.003157,0.000212,0.000076,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.000000e+00,2.871734e-10,2.871734e-10,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00


In [266]:
ns_accumulated_df_valence = pd.DataFrame()
ns_accumulated_df_conduction = pd.DataFrame()
for s_select in range(nS):
    accumulated_df_valence = pd.DataFrame()
    accumulated_df_conduction = pd.DataFrame()
    for k_select in range(len(processed_kgrid_wfn_eigenvector)):
        k_select_kpdos = int(processed_kgrid_wfn_eigenvector[k_select][3])
        k_select_eigenvector = int(processed_kgrid_wfn_eigenvector[k_select][4])
        current_weighted_sum_valence = reversed_valence_df.loc[k_select_kpdos+1].mul(contrib_v_dict[(s_select, k_select_eigenvector)], axis=0).sum()
        current_weighted_sum_conduction = conduction_df.loc[k_select_kpdos+1].mul(contrib_c_dict[(s_select, k_select_eigenvector)], axis=0).sum()
        if accumulated_df_valence.empty:
            accumulated_df_valence = pd.DataFrame([current_weighted_sum_valence])
        else:
            accumulated_df_valence += current_weighted_sum_valence
        if accumulated_df_conduction.empty:
            accumulated_df_conduction = pd.DataFrame([current_weighted_sum_conduction])
        else:
            accumulated_df_conduction += current_weighted_sum_conduction
    ns_accumulated_df_valence = pd.concat([ns_accumulated_df_valence, accumulated_df_valence], axis=0)
    ns_accumulated_df_conduction = pd.concat([ns_accumulated_df_conduction, accumulated_df_conduction], axis=0)

print(accumulated_df_conduction)
#result_df
ns_accumulated_df_conduction


         0         1         2    3    4    5    6    7    8    9    ...  \
0  72.028354  6.579028  2.975912  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   

            674           675           676           677           678  \
0  1.147920e-07  1.148142e-07  7.758900e-07  7.758834e-07  1.800355e-07   

            679           680           681           682           683  
0  6.601036e-08  6.604665e-08  1.793859e-07  1.147383e-07  1.146511e-07  

[1 rows x 684 columns]


,0,1,2,3,4,5,6,7,8,9,...,674,675,676,677,678,679,680,681,682,683
0,175.649257,17.020247,2.949807,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.439492e-06,3.447366e-06,2.404518e-05,2.404417e-05,5.162654e-06,1.927857e-06,1.930353e-06,5.153377e-06,3.450020e-06,3.452785e-06
0,136.057263,13.041550,2.936302,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.837649e-06,2.842923e-06,1.927369e-05,1.927267e-05,4.217610e-06,1.598127e-06,1.599827e-06,4.203800e-06,2.850162e-06,2.846836e-06
0,262.872183,25.764880,2.951560,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.007901e-06,3.013771e-06,2.180472e-05,2.180315e-05,4.570123e-06,1.627677e-06,1.632081e-06,4.560038e-06,3.016928e-06,3.019598e-06
0,222.959365,21.758290,2.928101,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.134691e-06,4.136776e-06,2.703423e-05,2.703045e-05,6.607636e-06,2.087117e-06,2.092546e-06,6.533831e-06,4.165945e-06,4.130374e-06
0,198.801825,19.351550,3.157442,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.139398e-05,1.141173e-05,8.585535e-05,8.585448e-05,1.757733e-05,8.209252e-06,8.218083e-06,1.757310e-05,1.141282e-05,1.141840e-05
0,188.159759,18.284678,3.121035,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.292833e-06,9.310562e-06,7.039666e-05,7.039618e-05,1.429972e-05,6.789579e-06,6.796089e-06,1.429549e-05,9.311417e-06,9.315188e-06
0,209.799645,20.457776,3.115917,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.666328e-06,8.683708e-06,6.526269e-05,6.526191e-05,1.328649e-05,6.131793e-06,6.137701e-06,1.328043e-05,8.683630e-06,8.687943e-06
0,209.279926,20.387570,3.275107,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.548174e-05,1.551107e-05,1.315679e-04,1.315665e-04,2.486891e-05,1.132389e-05,1.133781e-05,2.486444e-05,1.551237e-05,1.551899e-05
0,70.816531,6.447213,2.988341,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.434529e-08,9.437522e-08,6.346652e-07,6.346607e-07,1.485399e-07,5.388623e-08,5.391417e-08,1.480514e-07,9.431856e-08,9.427483e-08
0,72.028354,6.579028,2.975912,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.147920e-07,1.148142e-07,7.758900e-07,7.758834e-07,1.800355e-07,6.601036e-08,6.604665e-08,1.793859e-07,1.147383e-07,1.146511e-07


In [267]:
ns_accumulated_df_valence

,0,1,2,3,4,5,6,7,8,9,...,674,675,676,677,678,679,680,681,682,683
0,178.416930,17.287026,-0.340467,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,138.034834,13.227484,-0.336249,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,269.381049,26.404652,-0.329777,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,228.700185,22.319461,-0.323561,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,200.491502,19.514490,-0.374531,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,189.933186,18.455101,-0.368759,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,212.312932,20.702033,-0.371864,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,209.598305,20.418239,-0.467315,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,73.134824,6.656179,-0.356767,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,74.492674,6.802828,-0.352518,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [268]:
max(accumulated_df_conduction.iloc[:,4:].sum(axis=0))

0.131316383047232

In [269]:
# Save comments and ns_accumulated_df_valence to a file, with every row in df is a single line
with open('pdos_sum_valence.dat', 'w') as f:
    for line in comments:
        f.write(line)
    for i in range(len(ns_accumulated_df_valence)):
        f.write(str(ns_accumulated_df_valence.iloc[i,:].values.tolist()).strip('[]') + '\n')


with open('pdos_sum_conduction.dat', 'w') as f:
    for line in comments:
        f.write(line)
    for i in range(len(ns_accumulated_df_conduction)):
        f.write(str(ns_accumulated_df_conduction.iloc[i,:].values.tolist()).strip('[]') + '\n')

## Step 4: Calculate the projectiong into different part of the heterostructure

This part is system specific, for now I just hard-coded it

### 4.0 for APD1_Na

In [270]:
# columns from 37 to 72, 361 to 424 as the Br-per. contribution
# columns from 1 to 36, 73 to 360 as the Br-non. contribution
# columns from 425 to 488 as the O contribution
# columns from 609 to 648 as the Na contribution
# Total is 1 to 680

conduction_perovskite = ns_accumulated_df_conduction.iloc[:, 37+3:73+3].sum(axis=1) + ns_accumulated_df_conduction.iloc[:, 361+3:425+3].sum(axis=1)
conduction_non_perovskite = ns_accumulated_df_conduction.iloc[:, 1+3:37+3].sum(axis=1) + ns_accumulated_df_conduction.iloc[:, 73+3:361+3].sum(axis=1)
conduction_O = ns_accumulated_df_conduction.iloc[:, 425+3:489+3].sum(axis=1)
conduction_Na = ns_accumulated_df_conduction.iloc[:, 609+3:649+3].sum(axis=1)
conduction_total = ns_accumulated_df_conduction.iloc[:, 1+3:681+3].sum(axis=1)
conduction_other = ns_accumulated_df_conduction.iloc[:, 489+3:609+3].sum(axis=1) + ns_accumulated_df_conduction.iloc[:, 649+3:681+3].sum(axis=1)
conduction_others = conduction_total - conduction_perovskite - conduction_non_perovskite - conduction_O - conduction_Li - conduction_other

In [271]:
conduction_perovskite/ conduction_total

0    0.572081
0    0.540219
0    0.509864
0    0.503027
0    0.307781
0    0.308798
0    0.313290
0    0.290306
0    0.244785
0    0.282085
dtype: float64

In [272]:
conduction_non_perovskite/ conduction_total

0    0.381534
0    0.411311
0    0.438636
0    0.446526
0    0.615782
0    0.618665
0    0.614834
0    0.613885
0    0.678148
0    0.645075
dtype: float64

In [273]:
# valence bands for APD1_Na
valence_perovskite = ns_accumulated_df_valence.iloc[:, 37+3:73+3].sum(axis=1) + ns_accumulated_df_valence.iloc[:, 361+3:425+3].sum(axis=1)
valence_non_perovskite = ns_accumulated_df_valence.iloc[:, 1+3:37+3].sum(axis=1) + ns_accumulated_df_valence.iloc[:, 73+3:361+3].sum(axis=1)
valence_O = ns_accumulated_df_valence.iloc[:, 425+3:489+3].sum(axis=1)
valence_Na = ns_accumulated_df_valence.iloc[:, 609+3:649+3].sum(axis=1)
valence_total = ns_accumulated_df_valence.iloc[:, 1+3:681+3].sum(axis=1)
valence_other = ns_accumulated_df_valence.iloc[:, 489+3:609+3].sum(axis=1) + ns_accumulated_df_valence.iloc[:, 649+3:681+3].sum(axis=1)
valence_others = valence_total - valence_perovskite - valence_non_perovskite - valence_O - valence_Li - valence_other

In [274]:
valence_non_perovskite/ valence_total

0    0.247267
0    0.261333
0    0.263482
0    0.267242
0    0.305778
0    0.307863
0    0.310630
0    0.261201
0    0.335529
0    0.321984
dtype: float64

In [275]:
valence_perovskite/ valence_total

0    0.396037
0    0.363208
0    0.357579
0    0.344821
0    0.263010
0    0.258879
0    0.255903
0    0.276277
0    0.155830
0    0.188574
dtype: float64

In [276]:
valence_O/ valence_total

0    0.356691
0    0.375456
0    0.378936
0    0.387934
0    0.431145
0    0.433199
0    0.433377
0    0.462389
0    0.508641
0    0.489442
dtype: float64

### 4.1 for APD2_Pb

In [200]:
# columns from 37 to 72, 105 to 168 as the Br-per. contribution
# columns from 1 to 36, 73 to 104 as the Br-non. contribution
# columns from 169 to 232 as the O contribution
# columns from 609 to 624 as the Li contribution
# Total is 1 to 656

conduction_perovskite = ns_accumulated_df_conduction.iloc[:, 37+3:73+3].sum(axis=1) + ns_accumulated_df_conduction.iloc[:, 105+3:169+3].sum(axis=1)
conduction_non_perovskite = ns_accumulated_df_conduction.iloc[:, 1+3:37+3].sum(axis=1) + ns_accumulated_df_conduction.iloc[:, 73+3:105+3].sum(axis=1)
conduction_O = ns_accumulated_df_conduction.iloc[:, 169+3:233+3].sum(axis=1)
conduction_Li = ns_accumulated_df_conduction.iloc[:, 609+3:625+3].sum(axis=1)
conduction_total = ns_accumulated_df_conduction.iloc[:, 4:660].sum(axis=1)
conduction_other = ns_accumulated_df_conduction.iloc[:, 233+3:609+3].sum(axis=1) + ns_accumulated_df_conduction.iloc[:, 625+3:657+3].sum(axis=1)
conduction_others = conduction_total - conduction_perovskite - conduction_non_perovskite - conduction_O - conduction_Li - conduction_other

In [201]:
conduction_perovskite/ conduction_total

0    0.999966
0    0.999962
0    0.999911
0    0.999907
0    0.999997
0    0.999997
0    0.999996
0    0.999997
0    0.999997
0    0.999997
dtype: float64

In [202]:
valence_perovskite = ns_accumulated_df_valence.iloc[:, 37+3:73+3].sum(axis=1) + ns_accumulated_df_valence.iloc[:, 105+3:169+3].sum(axis=1)
valence_non_perovskite = ns_accumulated_df_valence.iloc[:, 1+3:37+3].sum(axis=1) + ns_accumulated_df_valence.iloc[:, 73+3:105+3].sum(axis=1)
valence_O = ns_accumulated_df_valence.iloc[:, 169+3:233+3].sum(axis=1)
valence_Li = ns_accumulated_df_valence.iloc[:, 609+3:625+3].sum(axis=1)
valence_total = ns_accumulated_df_valence.iloc[:, 5:660].sum(axis=1)
valence_other = ns_accumulated_df_valence.iloc[:, 233+3:609+3].sum(axis=1) + ns_accumulated_df_valence.iloc[:, 625+3:657+3].sum(axis=1)
valence_others = valence_total - valence_perovskite - valence_non_perovskite - valence_O - valence_Li - valence_other

In [203]:
valence_non_perovskite/valence_total

0    0.393530
0    0.385594
0    0.410145
0    0.402179
0    0.417120
0    0.433687
0    0.407465
0    0.411081
0    0.423705
0    0.415607
dtype: float64

In [204]:
valence_Li/valence_total

0    3.742041e-10
0    3.961451e-10
0    2.864154e-10
0    3.041153e-10
0    7.297764e-11
0    1.322196e-11
0    9.618686e-11
0    5.564608e-11
0    9.889452e-12
0    5.686291e-11
dtype: float64

In [205]:
valence_O/valence_total

0    0.403635
0    0.410420
0    0.393557
0    0.400801
0    0.390073
0    0.379930
0    0.395956
0    0.398653
0    0.391470
0    0.391753
dtype: float64

In [206]:
valence_other/valence_total

0    0.202529
0    0.203659
0    0.196066
0    0.196771
0    0.192777
0    0.186364
0    0.196544
0    0.190237
0    0.184805
0    0.192605
dtype: float64

### 4.2 for Hypothetical COCN - Pb-Pb Intergrowth

In [140]:
# columns from 1 to 36, 173 to 236 as the Br-per. contribution
# columns from 37 to 172 as the Br-non. contribution
# columns from 237 to 268 as the O contribution
# columns from 269 to 428 as the other contribution
# Total is 1 to 428


conduction_perovskite = ns_accumulated_df_conduction.iloc[:, 1+3:37+3].sum(axis=1) + ns_accumulated_df_conduction.iloc[:, 173+3:237+3].sum(axis=1)
conduction_non_perovskite = ns_accumulated_df_conduction.iloc[:, 37+3:173+3].sum(axis=1)
conduction_O = ns_accumulated_df_conduction.iloc[:, 237+3:269+3].sum(axis=1)
conduction_other = ns_accumulated_df_conduction.iloc[:, 269+3:429+3].sum(axis=1)
conduction_total = ns_accumulated_df_conduction.iloc[:, 4:432].sum(axis=1)

In [141]:
conduction_non_perovskite/ conduction_total

0    0.999985
0    0.999984
0    0.999985
0    0.999985
0    0.999999
0    0.999999
0    0.999999
0    0.999999
0    0.999999
0    0.999999
dtype: float64

In [142]:
valence_perovskite = ns_accumulated_df_valence.iloc[:, 1+3:37+3].sum(axis=1) + ns_accumulated_df_valence.iloc[:, 173+3:237+3].sum(axis=1)
valence_non_perovskite = ns_accumulated_df_valence.iloc[:, 37+3:173+3].sum(axis=1)
valence_O = ns_accumulated_df_valence.iloc[:, 237+3:269+3].sum(axis=1)
valence_other = ns_accumulated_df_valence.iloc[:, 269+3:429+3].sum(axis=1)
valence_total = ns_accumulated_df_valence.iloc[:, 5:432].sum(axis=1)

In [143]:
valence_perovskite/valence_total

0    0.999999
0    0.999999
0    0.999999
0    0.999999
0    1.000000
0    1.000000
0    1.000000
0    1.000000
0    1.000000
0    1.000000
dtype: float64

### 4.3 for Hypothetical HOCN - Pb-Pb Intergrowth

In [95]:
# columns from 1 to 36, 173 to 236 as the Br-per. contribution
# columns from 37 to 172 as the Br-non. contribution
# columns from 237 to 268 as the O contribution
# columns from 269 to 380 as the Li contribution
# Total is 1 to 380


conduction_perovskite = ns_accumulated_df_conduction.iloc[:, 1+3:37+3].sum(axis=1) + ns_accumulated_df_conduction.iloc[:, 173+3:237+3].sum(axis=1)
conduction_non_perovskite = ns_accumulated_df_conduction.iloc[:, 37+3:173+3].sum(axis=1)
conduction_O = ns_accumulated_df_conduction.iloc[:, 237+3:269+3].sum(axis=1)
conduction_other = ns_accumulated_df_conduction.iloc[:, 269+3:381+3].sum(axis=1)
conduction_total = ns_accumulated_df_conduction.iloc[:, 4:381].sum(axis=1)


valence_perovskite = ns_accumulated_df_valence.iloc[:, 1+3:37+3].sum(axis=1) + ns_accumulated_df_valence.iloc[:, 173+3:237+3].sum(axis=1)
valence_non_perovskite = ns_accumulated_df_valence.iloc[:, 37+3:173+3].sum(axis=1)
valence_O = ns_accumulated_df_valence.iloc[:, 237+3:269+3].sum(axis=1)
valence_other = ns_accumulated_df_valence.iloc[:, 269+3:381+3].sum(axis=1)
valence_total = ns_accumulated_df_valence.iloc[:, 5:381].sum(axis=1)

In [96]:
conduction_non_perovskite/ conduction_total

0    0.533164
0    0.502276
0    0.472682
0    0.465814
0    0.284187
0    0.283479
0    0.287644
0    0.274745
0    0.227366
0    0.261032
dtype: float64

In [97]:
valence_perovskite/valence_total

0    0.207554
0    0.219934
0    0.222114
0    0.224684
0    0.257056
0    0.258786
0    0.260187
0    0.217740
0    0.285151
0    0.269968
dtype: float64